# Evaluate on Office-Home dataset

# Imports

In [1]:
import numpy as np
import torch
from torchvision import transforms as T
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter

import sys
sys.path.append("..")

from data import office_home
import models.resnet_ms as resnet_ms

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/119M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/791M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15588 [00:00<?, ? examples/s]

# Config

## Regarding Dataset

In [2]:
NUM_CLASSES = 65
CLASSES = ["Alarm Clock", "Backpack", "Batteries", "Bed", "Bike", "Bottle", "Bucket", "Calculator", "Calendar", "Candles", "Chair", "Clipboards", "Computer", "Couch", "Curtains", "Desk Lamp", "Drill", "Eraser", "Exit Sign", "Fan", "File Cabinet", "Flipflops", "Flowers", "Folder", "Fork", "Glasses", "Hammer", "Helmet", "Kettle", "Keyboard", "Knives",
"Lamp Shade", "Laptop", "Marker", "Monitor", "Mop", "Mouse", "Mug", "Notebook", "Oven", "Pan", "Paper Clip", "Pen", "Pencil", "Postit Notes", "Printer", "Push Pin", "Radio", "Refrigerator", "Ruler", "Scissors", "Screwdriver", "Shelf", "Sink", "Sneakers", "Soda", "Speaker", "Spoon", "Table", "Telephone", "Toothbrush", "Toys", "Trash Can", "TV", "Webcam"]
DOMAINS = ["Art", "Clipart", "Product", "Real World"]

## Hyperparameters

In [14]:
EPOCHS = 25
BATCH_SIZE = 32
LEARNING_RATE = 1e-5
REGULARIZATION = 1e-3
MODEL = resnet_ms.resnet18
USE_PRETRAINED = True
OPTIMIZER = optim.AdamW
OPTIMIZER_KWARGS = {
    "lr": LEARNING_RATE, # Standard value for AdamW: 1e-3
    "weight_decay": REGULARIZATION # Standard value for AdamW: 1e-2
}
SCHEDULER = optim.lr_scheduler.ReduceLROnPlateau
SCHEDULER_KWARGS = {"mode": "min", "patience": 3}
EARLY_STOPPING_PATIENCE = 5
EARLY_STOPPING_DELTA = 1e-5
AUGMENTATIONS = ()
NUM_SEEDS = 1

## Image Normalization

In [15]:
# Values for pretrained ResNet
pretrained_image_transform = T.Compose([
    *AUGMENTATIONS,
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Device

In [16]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Device: {device}")

Device: cuda


## Abstract model building, optimizer and scheduler

In [17]:
build_model = lambda: MODEL(NUM_CLASSES, loss='softmax', pretrained=USE_PRETRAINED)
build_optimizer = lambda model: OPTIMIZER(model.parameters(), **OPTIMIZER_KWARGS)
build_scheduler = lambda optimizer: SCHEDULER(optimizer, **SCHEDULER_KWARGS)

# Set seed for reproducibility

In [18]:
# seed = 42
# torch.manual_seed(42)
# if device == torch.device("cuda"):
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True

# Training

In [19]:
writer = SummaryWriter()
%load_ext tensorboard
%tensorboard --logdir ./runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/hans/anaconda3/envs/xai_proj_t2/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/tensorboard/main.py", line 27, in <module>
    from tensorboard import default
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/tensorboard/default.py", line 33, in <module>
    from tensorboard.plugins.core import core_plugin
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/tensorboard/plugins/core/core_plugin.py", line 32, in <module>
    from tensorboard.util import grpc_util
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/tensorboard/util/grpc_util.py", line 24, in <module>
    import grpc
  File "/home/hans/anaconda3/envs/xai_proj_t2/lib/python3.11/site-packages/grpc/__init__.py", line 22, in <module

## Training Loop

In [20]:
class AverageMeter:
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [21]:
def accuracy(target, output):
    batch_size = target.shape[0]
    _, pred = torch.max(output, dim=-1)
    correct = pred.eq(target).sum()
    return correct.item() / batch_size

In [22]:
def train(epoch: int,
        target_domain: str,
        data_loader:torch.utils.data.DataLoader,
        model: nn.Module,
        optimizer: optim.Optimizer
        ) -> tuple[float, float]:
    """train one epoch"""
    model.train()
    losses = AverageMeter()
    accs = AverageMeter()

    for i, (data, target) in enumerate(data_loader):
        step = (epoch - 1) * len(data_loader) + i + 1
        data = data.to(device)
        target = target.to(device)

        out = model(data)
        loss = F.cross_entropy(out, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = accuracy(target, out)
        losses.update(loss.item(), out.shape[0])
        accs.update(acc, out.shape[0])

        writer.add_scalar(f'Loss/Train/target={target_domain}', loss.item(), step)
        writer.add_scalar(f'Accuracy/Train/target={target_domain}', acc, step)

    return losses.avg, accs.avg

## Evaluation

In [23]:
def evaluate(data_loader: torch.utils.data.DataLoader, model: nn.Module, phase="val") -> tuple[float, float]:
    model.eval()

    losses = AverageMeter()
    accs = AverageMeter()

    with torch.no_grad():
        for data, target in data_loader:
            data = data.to(device)
            target = target.to(device)

            out = model(data)

            # The implementation returns only the feature vector rather than the classification logits.
            # To compare the labels, we therefore must apply the classification layer manually:
            out = model.classifier(out)

            loss = F.cross_entropy(out, target)
            acc = accuracy(target, out)

            losses.update(loss.item(), out.shape[0])
            accs.update(acc, out.shape[0])
    
    return losses.avg, accs.avg

## Training Loop

In [24]:
all_results = {d: [] for d in DOMAINS}
all_results['avgs'] = []
all_results['worst'] = []

for _ in tqdm(range(NUM_SEEDS), desc="Seeds"):
    results = {}

    for target_domain in tqdm(DOMAINS, desc="Target Domain"):
        model = build_model()
        model = model.to(device)

        for name, param in model.named_parameters():
            if "classifier" not in name:
                param.requires_grad = False

        optimizer = build_optimizer(model)
        scheduler = build_scheduler(optimizer)

        if not USE_PRETRAINED:
            img_mean, img_std = office_home.get_normalization_stats(target_domain)
            print(f"Normalization values excluding domain {target_domain}:\n\tmean: {img_mean}\n\tstd: {img_std}")
            image_transform = T.Compose([
                *AUGMENTATIONS,
                T.Resize(256),
                T.CenterCrop(224),
                T.ToTensor(),
                T.Normalize(mean=img_mean, std=img_std)
            ])
        else:
            image_transform = pretrained_image_transform

        train_loader, test_loader = office_home.get_data_loaders(target_domain, train_batch_size=BATCH_SIZE, transform=image_transform, shuffle_test=True, drop_last=True)

        best_loss = float('inf')
        patience_counter = 0
        for epoch in tqdm(range(1, EPOCHS + 1), desc=f"Epoch ({target_domain})"):
            train_loss, train_acc = train(epoch, target_domain, train_loader, model, optimizer)
            test_loss, test_acc = evaluate(test_loader, model)

            writer.add_scalar(f"Loss/Test/target={target_domain}", test_loss, epoch)
            writer.add_scalar(f"Accuracy/Test/target={target_domain}", test_acc, epoch)

            scheduler.step(test_loss)

            if best_loss - test_loss < EARLY_STOPPING_DELTA and (patience_counter := patience_counter+1) > EARLY_STOPPING_PATIENCE:
                break

            if test_loss < best_loss:
                best_acc = test_acc
                torch.save(model.state_dict(), f"../checkpoints/mixstyle/best_{target_domain}.pt")

        model.load_state_dict(torch.load(f"../checkpoints/mixstyle/best_{target_domain}.pt"))
        _, acc = evaluate(test_loader, model, phase="final")

        results[target_domain] = acc

    avg_acc = np.mean([*results.values()])
    worst_case_acc = np.min([*results.values()])

    for d in DOMAINS:
        all_results[d].append(results[d])
    all_results['avgs'].append(avg_acc)
    all_results['worst'].append(worst_case_acc)

print("Average Accuracy:\n" +
      "{}".format("".join(f"\t{d}: {np.mean(all_results[d]):.4f}, std: {np.std(all_results[d]):.4f}\n" for d in DOMAINS)) +
      f"\ttotal: {np.mean(all_results['avgs']):.4f}, std: {np.std(all_results['avgs']):.4f}\n"
      "Worst-case Accuracy:\n" +
      "{}".format("".join(f"\t{d}: {np.min(all_results[d]):.4f}\n" for d in DOMAINS)) +
      f"\ttotal: {np.mean(all_results['worst']):.4f}, std: {np.std(all_results['worst']):.4f}"
)

Seeds:   0%|          | 0/1 [00:00<?, ?it/s]

Target Domain:   0%|          | 0/4 [00:00<?, ?it/s]

Epoch (Art):   0%|          | 0/25 [00:00<?, ?it/s]

Epoch (Clipart):   0%|          | 0/25 [00:00<?, ?it/s]

Epoch (Product):   0%|          | 0/25 [00:00<?, ?it/s]

Epoch (Real World):   0%|          | 0/25 [00:00<?, ?it/s]

Average Accuracy:
	Art: 0.4847, std: 0.0000
	Clipart: 0.3670, std: 0.0000
	Product: 0.5894, std: 0.0000
	Real World: 0.6101, std: 0.0000
	total: 0.5128, std: 0.0000
Worst-case Accuracy:
	Art: 0.4847
	Clipart: 0.3670
	Product: 0.5894
	Real World: 0.6101
	total: 0.3670, std: 0.0000


In [ ]:
print("Average Accuracy:\n" +
      "{}".format("".join(f"\t{d}: {np.mean(all_results[d]):.4f}, std: {np.std(all_results[d]):.4f}\n" for d in DOMAINS)) +
      f"\ttotal: {np.mean(all_results['avgs']):.4f}, std: {np.std(all_results['avgs']):.4f}\n"
      "Worst-case Accuracy:\n" +
      "{}".format("".join(f"\t{d}: {np.min(all_results[d]):.4f}\n" for d in DOMAINS)) +
      f"\ttotal: {np.mean(all_results['worst']):.4f}, std: {np.std(all_results['worst']):.4f}"
      )

Average Accuracy:
	art_painting: 0.8934, std: 0.0036
	cartoon: 0.7596, std: 0.0041
	photo: 0.9774, std: 0.0030
	sketch: 0.7298, std: 0.0048
	total: 0.8400, std: 0.0033
Worst-case Accuracy:
	art_painting: 0.8887
	cartoon: 0.7564
	photo: 0.9736
	sketch: 0.7235
	total: 0.7298, std: 0.0048
